## **1. Téléchargement des données de test**

In [1]:
!wget https://nextcloud.ig.umons.ac.be/s/py2DDt6mscBgFAE/download/TestData.zip
!unzip TestData.zip -d test_data
!rm TestData.zip

--2025-10-23 22:11:33--  https://nextcloud.ig.umons.ac.be/s/py2DDt6mscBgFAE/download/TestData.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)... 193.190.209.220
Connecting to nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://nextcloud.ig.umons.ac.be/public.php/dav/files/py2DDt6mscBgFAE/?accept=zip [following]
--2025-10-23 22:11:33--  https://nextcloud.ig.umons.ac.be/public.php/dav/files/py2DDt6mscBgFAE/?accept=zip
Reusing existing connection to nextcloud.ig.umons.ac.be:443.
HTTP request sent, awaiting response... 200 OK
Length: 433130624 (413M) [application/zip]
Saving to: 'TestData.zip'

TestData.zip        100%[===================>] 413.07M  9.20MB/s    in 67s     

2025-10-23 22:12:41 (6.12 MB/s) - 'TestData.zip' saved [433130624/433130624]

Archive:  TestData.zip
   creating: test_data/TestData/
  i

## **2. Importation des librairies**

In [1]:
import torch
from PIL import Image
from pathlib import Path
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

# **3. Téléchargement de la BD de test**

In [2]:
img_size = 224
folder_path  = 'test_data/TestData'
model = torch.jit.load(os.path.join(os.getcwd(), "checkpoints/best_vgg_DB2_model_jit.pth"))

In [3]:
test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])

## **4. Générer le fichier CSV de prédictions pour soumission au challenge Kaggle**

In [4]:
def predict_folder(model, folder_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()


    predictions = []
    filenames = []

    for img_path in os.listdir(folder_path):
        with torch.no_grad():
            # Load and preprocess image
            image = Image.open(folder_path+'/'+img_path).convert('RGB')
            image = test_transform(image).unsqueeze(0).to(device)

            # Get prediction
            output = model(image)
            _, predicted = output.max(1)

            predictions.append(predicted.item())
            filenames.append(img_path)

    # Create submission dataframe
    submission_df = pd.DataFrame({
        'ID': filenames,
        'Label': predictions
    })

    # Save to CSV
    submission_df.to_csv('best_vgg_small_longlong_Dataset_longtrain_submission.csv', index=False)
    return submission_df


submission = predict_folder(model, folder_path)